In [24]:
import tensorflow as tf
from keras.models import *
from keras.layers import *
from keras.datasets import imdb
from keras.utils import pad_sequences

In [47]:
class TransformerBlock(Layer):
  def __init__(self,embed_dim, num_heads,ff_dim, rate=0.1):


    super().__init__()
    self.att= MultiHeadAttention(num_heads=num_heads,key_dim=embed_dim)
    self.ffn=Sequential(
        [Dense(ff_dim,activation='relu'),Dense(embed_dim),]
    )

    self.layernorm1=LayerNormalization(epsilon=1e-6)
    self.layernorm2=LayerNormalization(epsilon=1e-6)
    self.dropout1=Dropout(rate)
    self.dropout2=Dropout(rate)

  def call(self, inputs, training):
    attn_output = self.att(inputs,inputs)
    attn_output= self.dropout1(attn_output,training=training)

    out1= self.layernorm1(inputs+ attn_output)
    ffn_output= self.ffn(out1)
    ffn_output = self.layernorm1(inputs + attn_output)

    ffn_output = self.ffn(out1)
    ffn_output= self.dropout2(ffn_output,training=training)
    return self.layernorm2(out1+ffn_output)

In [48]:
class TokenAndPositionEmbedding(Layer):
  def __init__(self,maxlen,vocab_size,embed_dim):
    super().__init__()
    self.token_emb=Embedding(input_dim=vocab_size, output_dim=embed_dim)
    self.pos_emb= Embedding(input_dim=maxlen, output_dim=embed_dim)

  def call(self,x):
    maxlen=tf.shape(x)[-1]
    positions= tf.range(start=0,limit=maxlen,delta=1)
    positions= self.pos_emb(positions)
    x= self.token_emb(x)
    return x+positions

In [49]:
vocab_size = 20000 #only consider the top 20k
maxlen = 200 # only consider the first 200 words of each movie review
(x_train,y_train),(x_val,y_val) = imdb.load_data(num_words=vocab_size)
print(len(x_train),"training sequences")
print(len(x_val),"Validation sequences")
x_train = pad_sequences(x_train,maxlen = maxlen)
x_val = pad_sequences(x_val,maxlen=maxlen)

25000 training sequences
25000 Validation sequences


In [50]:
x_train

array([[   5,   25,  100, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]], dtype=int32)

In [51]:
x_train.shape

(25000, 200)

In [52]:
#flask assignment
# create UI for review model in CNN

In [54]:
embed_dim =32
num_heads=2
ff_dim=32

inputs=Input(shape=(maxlen,))
embedding_layer= TokenAndPositionEmbedding(maxlen,vocab_size, embed_dim)
x=embedding_layer(inputs)
transformer_block=TransformerBlock(embed_dim,num_heads,ff_dim)
x=transformer_block(x)
x=GlobalAveragePooling1D()(x)
x=Dropout(0.1)(x)
x= Dense(20, activation='relu')(x)
x=Dropout(0.1)(x)
outputs=Dense(2,activation='softmax')(x)

model= Model(inputs=inputs, outputs=outputs)

In [57]:
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
history=model.fit(x_train,y_train,batch_size=32,
                  epochs=10,
                  validation_data=(x_val,y_val))

Epoch 1/10
782/782 [==============================] - 107s 134ms/step - loss: 0.3764 - accuracy: 0.8242 - val_loss: 0.3003 - val_accuracy: 0.8728
Epoch 2/10
782/782 [==============================] - 125s 161ms/step - loss: 0.1925 - accuracy: 0.9278 - val_loss: 0.3735 - val_accuracy: 0.8482
Epoch 3/10
782/782 [==============================] - 105s 134ms/step - loss: 0.1294 - accuracy: 0.9532 - val_loss: 0.4218 - val_accuracy: 0.8497
Epoch 4/10
782/782 [==============================] - 116s 149ms/step - loss: 0.0860 - accuracy: 0.9697 - val_loss: 0.4765 - val_accuracy: 0.8462
Epoch 5/10
782/782 [==============================] - 128s 163ms/step - loss: 0.0600 - accuracy: 0.9806 - val_loss: 0.6002 - val_accuracy: 0.8477
Epoch 6/10
782/782 [==============================] - 105s 134ms/step - loss: 0.0402 - accuracy: 0.9871 - val_loss: 0.6839 - val_accuracy: 0.8368
Epoch 7/10
782/782 [==============================] - 104s 133ms/step - loss: 0.0324 - accuracy: 0.9900 - val_loss: 0.8321 -